# Group 10 - Project Phase 1
This notebook is the work and submission of Group 10 of CSMODEl Section S16. The group's members consist of:
* David, Peter Jan B.
* De Guzman, Evan Mari B.
* Manaois, Kyla Nicole G.
* Wangkay, Laurize Jeante G.


## Brief Description of the Dataset
The dataset being used for this study is the Spotify Top-2000s Mega Dataset. This dataset contains the Top 2000 songs in spotify, ranging from the years 1956 to 2019. This dataset has been acquired from Kaggle.com and was produced by the user Sumat Singh

### Data Collection Process
In collecting the data, the creator of this dataset used a third party website using this link, http://sortyourmusic.playlistmachinery.com/. This third party website uses the Spotify API to extract the data of a certain song and is then collected. The third party website was created by Paul Lamere.

### Structure of the Dataset
The structure of the dataset is quite simple. There exists 1994 observations or rows and there are 15 variables or columns. Each row is an entire set of information of a song, from its title and its artist, to its popularity. Each column represents a certain variable to be discussed further after this section. The only column that has no significance in the dataset is the index column attached with the dataset as to say that the index has no real merit in ranking the song. 

## Variables in the Dataset 
- **`Title`**: Name of the track.
- **`Artist`**: Name of the artist.
- **`Top Genres`**: Genre that the track applies to.
- **`Year`**: Release year of the track.
- **`Beats per Minute (BPM)`**: Tempo of the song.
- **`Energy`**: Energy of the song. A higher value pertains to the song being more energetic.
- **`Danceability`**: Danceability of the song. A higher value pertains to how easier it is to dance to a song.
- **`Loudness`**: Loudness of the song. A higher value pertains a louder song. 
- **`Valence`**: The positivity of a song. A higher value pertains to a more positive mood for the song.
- **`Length`**: The duration of the song.
- **`Acoustic`**: The acoustic value of the song. A higher value pertains that the song was made less electronically. 
- **`Speechiness`**: The presence of spoken words in the song. A higher value pertains that the song has more spoken words.
- **`Popularity`**: The popularity of a song. A higher value pertains to a more popular song.

## Data Cleaning


### Importing Libraries
For this section of the notebook, our main focus is simply cleaning the dataset. To fufill this purpose, it is deemed as a necessity to import the numpy and the pandas library.

In [2]:
import numpy as np
import pandas as pd

Then to load the dataset and view the first few rows with the use of `head()` function

In [3]:
spotify_df = pd.read_csv("Spotify-2000.csv")
spotify_df.head()

,Index,Title,Artist,Top Genre,Year,Beats Per Minute (BPM),Energy,Danceability,Loudness (dB),Liveness,Valence,Length (Duration),Acousticness,Speechiness,Popularity
0,1,Sunrise,Norah Jones,adult standards,2004,157,30,53,-14,11,68,201,94,3,71
1,2,Black Night,Deep Purple,album rock,2000,135,79,50,-11,17,81,207,17,7,39
2,3,Clint Eastwood,Gorillaz,alternative hip hop,2001,168,69,66,-9,7,52,341,2,17,69
3,4,The Pretender,Foo Fighters,alternative metal,2007,173,96,43,-4,3,37,269,0,4,76
4,5,Waitin' On A Sunny Day,Bruce Springsteen,classic rock,2002,106,82,58,-5,10,87,256,1,3,59


Now to view the general dataset information with the use of `info()` function

In [4]:
spotify_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1994 entries, 0 to 1993
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Index                   1994 non-null   int64 
 1   Title                   1994 non-null   object
 2   Artist                  1994 non-null   object
 3   Top Genre               1994 non-null   object
 4   Year                    1994 non-null   int64 
 5   Beats Per Minute (BPM)  1994 non-null   int64 
 6   Energy                  1994 non-null   int64 
 7   Danceability            1994 non-null   int64 
 8   Loudness (dB)           1994 non-null   int64 
 9   Liveness                1994 non-null   int64 
 10  Valence                 1994 non-null   int64 
 11  Length (Duration)       1994 non-null   object
 12  Acousticness            1994 non-null   int64 
 13  Speechiness             1994 non-null   int64 
 14  Popularity              1994 non-null   int64 
dtypes: i

Thanks to the `info()` function, we can see that there are 1994 observations and 15 columns. And within the same function, we get to see that there are exactly 1994 non-null items in every column. To double-check, this code is employed.

In [8]:
spotify_df.isnull().any()

Index                     False
Title                     False
Artist                    False
Top Genre                 False
Year                      False
Beats Per Minute (BPM)    False
Energy                    False
Danceability              False
Loudness (dB)             False
Liveness                  False
Valence                   False
Length (Duration)         False
Acousticness              False
Speechiness               False
Popularity                False
dtype: bool

Now to further check the correctness of the data, the Title and Artists should have capitalized strings. To do so, the group employed the use of a function to return a boolean checking if the first character of a string is capitalized or not.

In [30]:
def is_capitalized(s):
    return s == s.title()

is_title = spotify_df['Title'].apply(is_capitalized)
is_artist = spotify_df['Artist'].apply(is_capitalized)

capital_check = pd.DataFrame(
    {
    'Title' : is_title,
    'Artist' : is_artist
    }
)
capital_check

,Title,Artist
0,True,True
1,True,True
2,True,True
3,True,True
4,True,True
...,...,...
1989,True,True
1990,True,True
1991,True,True
1992,True,True


Now, there exists another possible problem in regards to the uniqueness of certain values. For example, the title "Hallelujah" has 3 different instances. Isolated within the variable title 

Here is a point of contention within the group for the use of the dataset within this section. The dataset is, 

## Exploratory Data Analysis

## Research Question